In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import vgg19

In [3]:
def output_img(image_path,target_height):
    original_width,original_height=keras.preprocessing.image.load_img(image_path).size
    target_width=int(original_width*target_height/original_height)
    return target_height,target_width

In [4]:
def preprocess_img(image_path,target_height,target_width):
    img=keras.preprocessing.image.load_img(image_path,target_size=(target_height,target_width))
    img_array=keras.preprocessing.image.img_to_array(img)
    img_array=np.expand_dims(img_array,axis=0)
    img_array=vgg19.preprocess_input(img_array)
    return tf.convert_to_tensor(img_array)

In [5]:
def build_model():
    model=vgg19.VGG19(weights='imagenet',include_top=False)
    outputs_dict=dict([(layer.name,layer.output)for layer in model.layers])
    return keras.Model(inputs=model.inputs,outputs=outputs_dict)

In [6]:
def create_optimizer():
    return keras.optimizers.Adam(
        keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=4.99,decay_steps=300,decay_rate=0.99
        )
    )
# switch with rmsprop next

In [7]:
image_height=512
num_iterations=5000

content_weight=8e-4
style_weight=8e-1

content_layer="block5_conv2"

style_layers=[
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
]

In [8]:
def gram_matrix(x):
    x=tf.transpose(x,(2,0,1))
    features=tf.reshape(x,(tf.shape(x)[0],-1))
    gram=tf.matmul(features,tf.transpose(features))
    return gram

In [9]:
def compute_content_loss(content_features,generated_features):
    original_img_features=content_features[content_layer]
    generated_img_features=generated_features[content_layer]
    return tf.reduce_sum(tf.square(generated_img_features-original_img_features))/2

In [10]:
def style_layer_loss(style_features,generated_features,image_size):
    style_gram=gram_matrix(style_features)
    generated_gram=gram_matrix(generated_features)
    num_channels=style_features.shape[2]
    return tf.reduce_sum(tf.square(style_gram-generated_gram))/(4.0*(num_channels**2)*(image_size**2))

In [11]:
def compute_style_loss(style_features,generated_features,image_size):
    total_style_loss=0
    for layer_name in style_layers:
        style_feature=style_features[layer_name][0]
        generated_feature=generated_features[layer_name][0]
        total_style_loss+=style_layer_loss(style_feature,generated_feature,image_size)/len(style_layers)
    return total_style_loss

In [12]:
def compute_loss(model,generated_img,content_features,style_features):
    generated_features=model(generated_img)
    content_loss_value=compute_content_loss(content_features,generated_features)
    style_loss_value=compute_style_loss(style_features,generated_features,generated_img.shape[1]*generated_img.shape[2])
    return content_weight*content_loss_value+style_weight*style_loss_value

In [13]:

def deprocess_image(tensor,height,width):
    tensor=tensor.numpy()
    tensor=tensor.reshape((height,width,3))
    tensor[:,:,0]+=103.939
    tensor[:,:,1]+=116.779
    tensor[:,:,2]+=123.680
    tensor=tensor[:,:,::-1]
    return np.clip(tensor,0,255).astype("uint8")

In [14]:

def save_image(generated_image,height,width,filename):
    img=deprocess_image(generated_image,height,width)
    keras.preprocessing.image.save_img(filename,img)

In [15]:
if __name__=="__main__":

    content_image_path='./pre/christopher-campbell-wtZgw1nQ3FI-unsplash.jpg'
    style_image_path='./pre/main-image.jpg'

    target_height,target_width=output_img(content_image_path,image_height)
    content_image=preprocess_img(content_image_path,target_height,target_width)
    style_image=preprocess_img(style_image_path,target_height,target_width)

    generated_image=tf.Variable(tf.random.uniform(style_image.shape,dtype=tf.dtypes.float32))
    model=build_model()
    optimizer=create_optimizer()

    # print(model.summary())
    content_features=model(content_image)
    style_features=model(style_image)
    
    for iteration in range(num_iterations):
        with tf.GradientTape() as tape:
            loss=compute_loss(model,generated_image,content_features,style_features)
        gradients=tape.gradient(loss,generated_image)
        print(f"Iteration: {iteration+1}, Loss: {loss:.2f}")
        optimizer.apply_gradients([(gradients,generated_image)])
        if(iteration+1)%100==0:
            filename=f"result/generated_at_iteration_{iteration+1}.png"
            save_image(generated_image,target_height,target_width,filename)
    filename=f"result/final_result_{num_iterations}_{content_weight}_{style_weight}.png"
    save_image(generated_image,target_height,target_width,filename)

Iteration: 1, Loss: 4688108.50
Iteration: 2, Loss: 4542411.50
Iteration: 3, Loss: 4145941.50
Iteration: 4, Loss: 3556267.75
Iteration: 5, Loss: 2893790.00
Iteration: 6, Loss: 2330124.75
Iteration: 7, Loss: 1986845.62
Iteration: 8, Loss: 1840018.38
Iteration: 9, Loss: 1714813.00
Iteration: 10, Loss: 1534572.25
Iteration: 11, Loss: 1354086.38
Iteration: 12, Loss: 1215739.50
Iteration: 13, Loss: 1121694.00
Iteration: 14, Loss: 1053704.25
Iteration: 15, Loss: 993193.50
Iteration: 16, Loss: 931414.12
Iteration: 17, Loss: 868126.31
Iteration: 18, Loss: 807801.25
Iteration: 19, Loss: 755145.94
Iteration: 20, Loss: 710903.06
Iteration: 21, Loss: 673674.00
Iteration: 22, Loss: 641941.00
Iteration: 23, Loss: 614490.19
Iteration: 24, Loss: 590255.38
Iteration: 25, Loss: 567875.88
Iteration: 26, Loss: 546634.31
Iteration: 27, Loss: 526146.44
Iteration: 28, Loss: 506367.25
Iteration: 29, Loss: 487828.91
Iteration: 30, Loss: 470607.50
Iteration: 31, Loss: 454879.31
Iteration: 32, Loss: 439874.44
Ite